In [2]:
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

import requests
import base64
from urllib.parse import urlencode, urlparse, parse_qs
import json
import datetime
from time import sleep

# Set up Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}

#import constants from KEYS module. create a module called KEYS.py and Add your own Client ID and Client Secret to it
import KEYS



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\alexr\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


In [2]:
#assign keys to values here

client_id = KEYS.CLIENT_ID
client_secret = KEYS.CLIENT_SECRET

In [3]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
    
    def get_client_credentiatls(self):
        """
        Returns a base64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_id == None or client_secret == None:
            raise Exception("You must set client_id and client_secret")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentiatls()
        return  {
            "Authorization" : f"Basic {client_creds_b64}",
        }
    
    def get_token_data(self):
        return {
            "grant_type":"client_credentials",
        }
    
    def perform_auth(self):
        token_url = self.token_url
        token_headers = self.get_token_headers()
        token_data = self.get_token_data()
        r = requests.post(token_url, headers=token_headers, data=token_data)
        if r.status_code not in range(200,299):
            raise Exception("Could not Authenticate client")
            #return False
        data = r.json() #convert json to dict
        now = datetime.datetime.now() # create datetime obj for now
        access_token = data['access_token'] #retrieve token from response
        expires_in = data['expires_in'] #retrieve token lifetime
        print(expires_in)
        expires = now + datetime.timedelta(seconds=expires_in) #creates datetime for when token expires
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < datetime.datetime.now() #boolean to rep if expired
        return True

    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token()
        return token

    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        return headers


    def get_resources(self, lookup_id, resource_type = 'albums', version = 'v1'):
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range (200,299):
            return {}
        return r.json()
    
    def get_user_permission(self, scopes):
        # open up code controlled Chrome Browser
        browser = Browser('chrome', **executable_path, headless=False)
        client_id  = self.client_id
        endpoint = "https://accounts.spotify.com/authorize"
        if not scopes:
            scopes = ['user-read-private','user-read-email']

        #create params dict for auth request
        params = {
            'client_id' : client_id,
            'scope': "%20".join(scopes),
            'response_type' : 'code',
            'redirect_uri' : 'https://example.com/callback',
            'state' : 'randomstatestring'
        }
        #create request
        url = endpoint + "?" + urlencode(params)

        #go to page and wait until user gives permission
        browser.visit(url)
        current_url = browser.url
        current_url_endpoint = current_url.split("?")[0]
        #wait until redirect page comes up
        while(current_url_endpoint != 'https://example.com/callback'):
            sleep(.5)
            current_url = browser.url
            current_url_endpoint = current_url.split("?")[0]

        #once and redirect uri, parse query paramemters out
        parsed_url = urlparse(browser.url)
        query_parameters = parse_qs(parsed_url.query)

        #close bowser
        browser.quit()

        if 'code' in query_parameters:
            return query_parameters['code'][0]
        else:
            print(query_parameters['error'][0])
            return None
        
    def get_user_auth_token(self, scopes):
        endpoint = 'https://accounts.spotify.com/api/token'
        headers = self.get_token_headers
        code = self.get_user_permission(scopes)
        headers.update({'Content-Type' : 'application/x-www-form-urlencoded'})
        body = {
            'grant_type' : 'authorization_code',
            'code' : code,
            'redirect_uri' : 'https://example.com/callback'
        }

        r = requests.post(endpoint, headers=headers, data = body)
        data = r.json() #convert json to dict
        now = datetime.datetime.now() # create datetime obj for now
        access_token = data['access_token'] #retrieve token from response
        expires_in = data['expires_in'] #retrieve token lifetime
        print(expires_in)
        expires = now + datetime.timedelta(seconds=expires_in) #creates datetime for when token expires
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < datetime.datetime.now() #boolean to rep if expired
        return True
        
        
    def create_public_playlist(self, playlist_name):
        pass
    
    def get_playlist(self, _id):
        return self.get_resources(_id, resource_type = 'playlist')


    def get_album(self, _id):
        return self.get_resources(_id, resource_type= 'albums')

    def get_artist(self, _id):
        return self.get_resources(_id, resource_type= 'artists')


    def base_search(self, query_params):
        headers = self.get_resource_header()
        
        endpoint = "https://api.spotify.com/v1/search"
        r = requests.get(endpoint, params=query_params, headers=headers)
         
        print(r.url)
        if r.status_code in range(200,299):
            return r.json()
        return {}

    def search(self, query=None, search_type = 'artist'):
        if query == None:
            raise Exception("A query is required")
        if isinstance(query, dict):
            query = " ".join([f'{k}:{v}' for k,v in query.items()])
        query_params = {
            "q":query,
            "type":search_type.lower()
        }
        
        return self.base_search(query_params)
    


In [17]:
# Set up Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\alexr\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


In [18]:
# Visit the Mars news site
url = 'https://accounts.spotify.com/authorize?client_id=af37f76b6c204b16b5f781ce1e54603d&response_type=code&redirect_uri=https%3A%2F%2Fexample.com%2Fcallback&scope=user-read-private%20user-read-email&state=34fFs29kd09'
browser.visit(url)

In [19]:
from urllib.parse import urlparse, parse_qs
parsed_url = urlparse(browser.url)
query_parameters = parse_qs(parsed_url.query)
print(query_parameters)
browser.quit()

{'code': ['AQDb44Afps2DOXTh6jF5E91UwHvSQgH-O0DkrSe13n5o8otqv_AG-dS3gtOUdOHmbL9gTt-bQ3jsX-tnlqxXEDkKPgnB4XCmvI5MWd3dhcjRXYNgvwsHobl8roxlEK1QmEq6hnbKZwIs23TuI3YDKD3bx8Er8TYRPoVDeup4WCw4RnGklAYKsTDFCRHwOm3JS2ZJ7QeLsO4wXX69lH4GxEg0Q0g'], 'state': ['34fFs29kd09']}


In [55]:
# open up code controlled Chrome Browser
browser = Browser('chrome', **executable_path, headless=False)
client_id  = 'af37f76b6c204b16b5f781ce1e54603d'
endpoint = "https://accounts.spotify.com/authorize"
scopes = ['playlist-modify-public']

#create params dict for auth request
params = {
    'client_id' : client_id,
    'scope': "%20".join(scopes),
    'response_type' : 'code',
    'redirect_uri' : 'https://example.com/callback',
    'state' : 'randomstatestring'
}
#create request
url = endpoint + "?" + urlencode(params)

#go to page and wait until user gives permission
browser.visit(url)
current_url = browser.url
current_url_endpoint = current_url.split("?")[0]
#wait until redirect page comes up
while(current_url_endpoint != 'https://example.com/callback'):
    sleep(.5)
    current_url = browser.url
    current_url_endpoint = current_url.split("?")[0]

#once and redirect uri, parse query paramemters out
parsed_url = urlparse(browser.url)
query_parameters = parse_qs(parsed_url.query)

#close bowser
browser.quit()


query_parameters['code'][0]


'AQBqJJoaK2zj80C-QGQ5vN5qtPdmULWnQUMW_L9fgicTCb9-OMAKOo8diiJHWvxYU6huhcWcBBVy1Llov7veSnmHXTNgIvNMxT9pOj5xkCsSqU-sebgfXqJDyzIsAcU-273uO5Ipe7SAVSgBiq6y4-483fCUljFZPi3U1NGcXaK7vH4GSNZOF2c6FnBD99GvkZxpIA8-Knk'

In [56]:
client_id = CLIENT_ID
client_secret = CLIENT_SECRET

endpoint = 'https://accounts.spotify.com/api/token'
headers = {}
code = query_parameters['code'][0]
headers.update({'Content-Type' : 'application/x-www-form-urlencoded'})
body = {
    'grant_type' : 'authorization_code',
    'code' : code,
    'redirect_uri' : 'https://example.com/callback',
    'client_id' : client_id,
    'client_secret' : client_secret
}

r = requests.post(endpoint, headers=headers, data = body)
        

In [57]:
auth_token_response = r.json()
print(auth_token_response)

{'access_token': 'BQCiWlZGfhccZyOWaNgE-fdeto0Be2PD3XyIOH_pDi2LecEYK8CPccUlOis7oHRlny35n7_2NAy5hpliQV-3aVAqrGy5wjIPWn4DAYWBaYSso8AlE0u_2qw1uLFXkpFWv33qubzS0_3ehw_ZykI-iLL5tkUXV9sOxcy7G1BwVMHwp0o', 'token_type': 'Bearer', 'expires_in': 3600, 'refresh_token': 'AQCpsmBr8_etPnJtJ1XGb2ClpJd6mTsbKp82ZDwAgOKqQudEbGA7mNuDiLMNDeiAVO-kalinhrZvKstlm44tLUG66g8jgvNg3lHNSsGXVqjBOgVQ5w0hWapv4sHpRIam-BQ', 'scope': 'playlist-modify-public'}


In [63]:
user_id = 'areta004'

authorize_token = f"Bearer {auth_token_response['access_token']}"

endpoint = f'https://api.spotify.com/v1/users/{user_id}/playlists'
headers = {
    "Authorization" : authorize_token,
    "Content-Type" : "application/json"
}
body = {
    "name":"A11 D4Y",
    "description":"A playlist with all the songs Girl Talk mixed into his album Feed the Animals",
    "public": True
}

r = requests.post(endpoint, headers=headers, json=body)

print(r.json())

{'collaborative': False, 'description': 'A playlist with all the songs Girl Talk mixed into his album Feed the Animals', 'external_urls': {'spotify': 'https://open.spotify.com/playlist/6aRhmtPeZusHaVehc6ZBQr'}, 'followers': {'href': None, 'total': 0}, 'href': 'https://api.spotify.com/v1/playlists/6aRhmtPeZusHaVehc6ZBQr', 'id': '6aRhmtPeZusHaVehc6ZBQr', 'images': [], 'name': 'F33D TH3 ANiMALZ', 'owner': {'display_name': 'areta004', 'external_urls': {'spotify': 'https://open.spotify.com/user/areta004'}, 'href': 'https://api.spotify.com/v1/users/areta004', 'id': 'areta004', 'type': 'user', 'uri': 'spotify:user:areta004'}, 'primary_color': None, 'public': True, 'snapshot_id': 'MSw1ODAxNDY2MTE0NTBiN2U0YzBjZWU4M2RjNWVhMTFkMzJhZTBhMDg1', 'tracks': {'href': 'https://api.spotify.com/v1/playlists/6aRhmtPeZusHaVehc6ZBQr/tracks', 'items': [], 'limit': 100, 'next': None, 'offset': 0, 'previous': None, 'total': 0}, 'type': 'playlist', 'uri': 'spotify:playlist:6aRhmtPeZusHaVehc6ZBQr'}


In [65]:
print(json.dumps(r.json(), indent = 4))

{
    "collaborative": false,
    "description": "A playlist with all the songs Girl Talk mixed into his album Feed the Animals",
    "external_urls": {
        "spotify": "https://open.spotify.com/playlist/6aRhmtPeZusHaVehc6ZBQr"
    },
    "followers": {
        "href": null,
        "total": 0
    },
    "href": "https://api.spotify.com/v1/playlists/6aRhmtPeZusHaVehc6ZBQr",
    "id": "6aRhmtPeZusHaVehc6ZBQr",
    "images": [],
    "name": "F33D TH3 ANiMALZ",
    "owner": {
        "display_name": "areta004",
        "external_urls": {
            "spotify": "https://open.spotify.com/user/areta004"
        },
        "href": "https://api.spotify.com/v1/users/areta004",
        "id": "areta004",
        "type": "user",
        "uri": "spotify:user:areta004"
    },
    "primary_color": null,
    "public": true,
    "snapshot_id": "MSw1ODAxNDY2MTE0NTBiN2U0YzBjZWU4M2RjNWVhMTFkMzJhZTBhMDg1",
    "tracks": {
        "href": "https://api.spotify.com/v1/playlists/6aRhmtPeZusHaVehc6ZBQr/track

In [83]:
playlist_id = r.json()["id"]
playlist_id

'6aRhmtPeZusHaVehc6ZBQr'

In [73]:
df = pd.read_csv('./data/tracks_uri.csv', index_col=0)
df

,artist,track,track_uri
0,Black Sabbath,War Pigs,spotify:track:2rd9ETlulTbz6BYZcdvIE1
1,The Pack,This Shit Slappin',spotify:track:4IeUAegXVbETb7FpgNDnSM
2,M.I.A.,Paper Planes,spotify:track:1ixbwbeBi5ufN4noUKmW5a
3,Jimmy Smith,I'm Gonna Love You Just a Little More Babe,spotify:track:1uu0es5Bu7orSWkqS5rXp7
4,Dorrough,Ice Cream Paint Job,spotify:track:57QFOU3f7O0gDLadxv5z8w
...,...,...,...
341,"Krave featuring Flo Rida, Pitbull, and Lil Jon",Go Crazy,spotify:track:2Y0wPrPQBrGhoLn14xRYCG
342,T.I.,Rubberband Man,spotify:track:6jN49cPLhJxsIUaoJ0yroP
343,Rosalind Rice featuring French Montana,Hustler,spotify:track:5ErtQGRKEibK2WHaA729O8
344,The Edgar Winter Group,Frankenstein,spotify:track:2l0Z9fY1E8Bep8opmhmDhq


In [86]:
#the playlist id comes from the response of creating the playlist
#playlist_id = r.json()["id"]
#use the same headers as before

for value in df['track_uri'].values:
    endpoint = f'https://api.spotify.com/v1/playlists/{playlist_id}/tracks'
    body = {'uris' : [value]}
    r = requests.post(endpoint, headers=headers, json=body)
    print(r.json())

{'snapshot_id': 'NywxOTJjNjNhYjc3YTlkODJlODVlZWNiY2FlZTIxYmE4Y2VjYjU2ZjAz'}
{'snapshot_id': 'OCxjNDZjMTJmODQyMzQ1YWI2ODE2Zjk2Yzk5NjJiMTIwZDg3NTczY2Jh'}
{'snapshot_id': 'OSxlMThjODJiMzNkNmRjZGExNzBlM2MwZjMxNDRiZGNiZGUzNTNhMmZm'}
{'snapshot_id': 'MTAsZTE0ZGRiOWI5NzNkOTAyYjA0ZWQzYzNiZTFlNmY3NWU3ZDkyMjM1NA=='}
{'snapshot_id': 'MTEsZGFlMTNkOWFiMGY4MTU5ZWY3ZDE0NmI2M2JiMzViNDBjNjY4OTlhZg=='}
{'snapshot_id': 'MTIsMWIzYWQ3ZmVjZDljOTRlYzY5ODM5MDY1ZDQ2ZjA0NWRkNThjMWFlYw=='}
{'snapshot_id': 'MTMsZGUwNTRkZTY0NjhkNjA2NzkwY2JlMTA3MDdmNGU0ZmQ5MDBiYjJhNA=='}
{'snapshot_id': 'MTQsODRiYjI0MTM2NDQ5MjllNTNlNjczNDRkNzRkNTI0YTE2NGQzZWU5NA=='}
{'snapshot_id': 'MTUsZGYyMWEwM2FlM2E4MWQzNmZmMjU5ZmUwZTY0NDFiOGQ3MDkyZGE2ZQ=='}
{'snapshot_id': 'MTYsNGI5OTE2MjNjMzg1YmI2OGI5OGQ5YjE5YzAxZWQyMmRmODIyNTAxOA=='}
{'snapshot_id': 'MTcsMjUzYzc2ZjU2YmZiOTI0M2Y1NzUzNzA0ZjQ1YWU1ZGM0ZTFkMjc0YQ=='}
{'snapshot_id': 'MTgsNWNlNTRhOGQwMWVmMmZhZTI3MzlmZTMxYjcyZmMxYTllYjRlYjg5NA=='}
{'snapshot_id': 'MTksZmM1MzY4YWI4N2VkYjg0NmI5OGU5MjA

{'snapshot_id': 'MTEwLDI0ZDczMjViMDQwMTc5YWMxNDM1NzU3NTljZDMxYTRjYWRiM2E5OWQ='}
{'snapshot_id': 'MTExLGVjNDA5ODY0OTFhMzcxNDRhNzJlYzY0ZmQwYTYwM2Q1YjUzMTMxOTU='}
{'snapshot_id': 'MTEyLDEzOTA1ZWY3M2Q4ZjdjZjY4YjVjZTU3MzVhMzIyMGRlOTQ5YzZiOTE='}
{'snapshot_id': 'MTEzLGU5MDA4MzkwYzIzOTBlMGQ5OTVkNWQyMDI1ZTIyZGY4MmNkMzMzMmE='}
{'snapshot_id': 'MTE0LDYwN2Q2YjBlZTZkMDA4M2JjYWRiYmMwNTA1MjBiYzdkZGQ0N2ZlOTM='}
{'snapshot_id': 'MTE1LGVmYzExNTNhMjQyMjIzN2JmNTExMDAzY2IxZWRmM2EzNGJlNzc5Yjc='}
{'snapshot_id': 'MTE2LGJiYzM1YTkzZjE2NTY2MDliZTk1YjA5ODQ4Y2ZlY2EzMjdhNjA1Mjk='}
{'snapshot_id': 'MTE3LGI3MWM3YjY2YTViNDAxZjliOGY3Y2NmZWYwZTMxMzU5NmY4MGExYjM='}
{'snapshot_id': 'MTE4LGMzYjZiODVkNGMwMDRhNmIwZjIzMjM5Zjk3YjE2N2ZiNjE5YmRhMGY='}
{'snapshot_id': 'MTE5LDBiOWY2NWZkNDNhYzliOTY5NGVlYWM0NDMzZGE5NWE5MGFlZjkzYmY='}
{'snapshot_id': 'MTIwLDUwZTZmZjgwZTY5MzNmZGVjM2VhN2I5ZmFiMzY3NjI3N2I1MDY4YzU='}
{'snapshot_id': 'MTIxLGFmZWRiZDI2MzNmMWJhYWM2MzBhM2YxMDc0NTAxZTYwNTEyMWE0ZDQ='}
{'snapshot_id': 'MTIyLDNjOWQxNDlmNjg3MjZ

{'snapshot_id': 'MjEzLDgwMTNmYTQ1MTllZjc3ZmRjYzY4NzIyM2IwNzM5MWY4NTBjNGQ5YjE='}
{'snapshot_id': 'MjE0LDYyZjIwOGNhOTUyNjUxNzRjMDg5M2RiYzJkNzBkZjlmMzc0NDM2NGM='}
{'snapshot_id': 'MjE1LDZmMTliNWQ5YmRkZTZjYTIwOGY0MjAyYjkwN2MzMmU3Y2QzZDg1MjU='}
{'snapshot_id': 'MjE2LDE1N2M5NTgxMmIxNjA4YmFkYzliOWVhYTFlZTNjYTc5OTdmN2U1MzY='}
{'snapshot_id': 'MjE3LDc3MzQ1ZTYwNTZhYTA4NjMxNWIxY2ZlYjc1NTlhZDBhYTM5Nzc3ZTA='}
{'snapshot_id': 'MjE4LDI5Y2UyOWNjMGVjZTIyOTM0ZTIwYjQ3NzliODQyOWI2NGM0ZThhMDk='}
{'snapshot_id': 'MjE5LGJmMGZjOTA0ZmRkMjM5OTFiZmE3NjRlZThlYTAzYmZjZTZiNzUxOGQ='}
{'snapshot_id': 'MjIwLGQzZDhkNjVmY2I1Zjk2YjIwMzNjMjMxN2I5ZDU4OTJkNTY1ODE3MmI='}
{'snapshot_id': 'MjIxLDJmODZkNDE0ZDFjZTdkMGY4YjNlZjZkNDkxMWI5NjY4Zjk4MmNkNmI='}
{'snapshot_id': 'MjIyLDUxM2QxOTVjNzhmY2Y0YWNiZDgxMzMxYmMzYzVjMzlkMDRlZWFjOWE='}
{'snapshot_id': 'MjIzLDMwODhlMTM4MWNlYjczOWMxNjM1MGMwZGZkYjNmMGZiY2ZjNDkxNjE='}
{'snapshot_id': 'MjI0LDQ4MTQ2YTIxNWI0YzE2N2FhNjlmZTkwMWQzMmI5NWNiMDMyYzk1ZTI='}
{'snapshot_id': 'MjI1LGI0YzZkYmRlYTJjMjY

{'snapshot_id': 'MzE3LGVkOTg1N2U4YTZhNDIzNzUzNzI4NDFlNzNkNGRhODlkMTdkMDQyZTg='}
{'snapshot_id': 'MzE4LDYwMDNiZjc0MjJlNWNmMDkyMjZiMGM0Yzg0MjM0NmY1MmRhODE4M2M='}
{'snapshot_id': 'MzE5LDI0NjgzYzU5OGNhMGVkNTdhODRlYzJlYjQ3MjllOTEzODYzY2EyN2E='}
{'snapshot_id': 'MzIwLGUyOGFhN2VhNDkwYWM5ZGMxODA5MzMwMDRhYzkxYjY1OGQzZTUzMjA='}
{'snapshot_id': 'MzIxLDcxYWJiMTJjNjdiNzU0ZWY3ZGIwNjAzMWJkMjFhODlkZGY4NGRlMWY='}
{'snapshot_id': 'MzIyLGIzYzJlOTdhYmE2YzMxNDk4MzQzMjdiMGY5ZTEzOTdkNzA3Y2JiMmM='}
{'snapshot_id': 'MzIzLDI1YzZjNGIwMmM5MWQ5N2YxNzkwMzc2OTkwOGZlZWI0OGNlNjgwM2M='}
{'snapshot_id': 'MzI0LGJmMGRjMmZmMTAwOGQwY2EzNjY3NGMxMWE4ZGIxNTBjMjA4MTkxYTk='}
{'snapshot_id': 'MzI1LDZlZjU2NDQxY2M2MmI2MmU2Y2JjOTA0ZmMxMTk2M2E3YTBmNjAxOTA='}
{'snapshot_id': 'MzI2LDY1ZDA1ZGY5YWE1NWVmOTVjMzRlNGM3ZjMyYTc1YTgwOTZmMTA2ZDI='}
{'snapshot_id': 'MzI3LGQ5NmYwODE2ZDBhMTFmODgwMmUxNDE0NjFjZmJkN2VjY2Q4MDI3MzM='}
{'snapshot_id': 'MzI4LGVlODY4NDdiZTMyNTNlZjBmNjdiZmI3ODg5YmZiNzIxODE4NmQzYjU='}
{'snapshot_id': 'MzI5LGM1NGY0NzZhMzJhNmV